# Import data and prepare dataset

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
from datasets import load_dataset,Dataset,DatasetDict, load_metric


t = pd.read_excel(f'{sdir}t4m_2022_05_05__21_59_v3.xlsx')
cnames = list(set(t['class_name'].values))
idx2cls = {i:v for i,v in enumerate(cnames)}
cls2idx = {v:i for i,v in enumerate(cnames)}

df = t[['content', 'text_1', 'text_2', 'text_3', 'text_5', 'class_name']].copy()
df['label'] = df['class_name'].map(cls2idx)
df['label'].value_counts()

tmp = df[['content', 'text_1', 'text_2', 'text_3', 'text_5','label']].copy()


data2 = tmp.copy()


data2 = data2[['content', 'text_1', 'text_2', 'text_3', 'text_5', 'label']].copy()
data2=Dataset.from_pandas(data2)

# 80% train, 20% test + validation
train_testvalid = data2.train_test_split(test_size=0.3,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

# gather everything to have a single DatasetDict
data2 = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train'],
                   })

tfidf_vectorizer = TfidfVectorizer() 

Xy_train = train_testvalid['train'].to_pandas()
Xy_test = test_valid['test'].to_pandas()
Xy_val = test_valid['train'].to_pandas()



# explore wrong labels and modify to multilabel

In [11]:
def fit_model_on_text(target, out=False):
    
    '''
    trai model, than add random string to each row, 
    and predict probability
    we'll get not the same input as on train, 
    and try to extract second problem
    '''
    
    tfidf_vectorizer = TfidfVectorizer() 
    X_train, y_train = Xy_train[target].copy(), Xy_train['label'].copy()
    X_val, y_val = Xy_val[target].copy(), Xy_val['label'].copy()
    X_test, y_test = Xy_test[target].copy(), Xy_test['label'].copy()

    X_train = pd.concat([X_train, X_val, X_test], ignore_index=True)
    y_train = y_train.values.tolist() + y_val.values.tolist() + y_test.values.tolist()

    tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
    
    X_test = X_train.copy()
    X_test += ' appstore google play бургер кинг' # random insert at the end

    tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

    classifier = RandomForestClassifier()

    classifier.fit(tfidf_train_vectors,y_train)

    y_pred = classifier.predict(tfidf_test_vectors)
    if out: print(classification_report(y_train,y_pred))
    y_pred = classifier.predict_proba(tfidf_test_vectors)
    

    return X_train, y_pred, y_train


# get classification report and dataset with predictions after random insert

In [20]:
X_train, y_pred, y_train = fit_model_on_text('text_3', out=1)

content = Xy_train['content'].values.tolist() +Xy_val['content'].values.tolist() +Xy_test['content'].values.tolist()
rv = pd.DataFrame({'content':content, 'target':X_train.values, 'y_pred':list(y_pred), 'true':y_train, 'y_label':[np.argmax(i) for i in y_pred]})

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       250
           1       0.99      0.98      0.98       168
           2       0.99      0.97      0.98       246
           3       0.93      0.97      0.95       215
           4       0.98      0.98      0.98       183
           5       0.90      0.98      0.94       293
           6       0.98      0.97      0.98       223
           7       0.98      0.98      0.98       300
           8       0.99      0.99      0.99       300
           9       0.94      0.97      0.96       151
          10       0.96      0.91      0.94       300
          11       0.99      0.98      0.99       120
          12       0.99      0.97      0.98       156
          13       1.00      0.93      0.96       116
          14       0.99      0.99      0.99       157

    accuracy                           0.97      3178
   macro avg       0.98      0.97      0.97      3178
weighted avg       0.97   

# this sections adds second class (true_2)

In [21]:
n = 2
thresh = 0.079

rvc = rv.copy()
rvc['true'] = rvc['true'].map(idx2cls)
rvc['y_label'] = rvc['y_label'].map(idx2cls)

# problems thad describe similar problems are groupped
clnames = set(cls2idx.keys())
syn = {
    'аккаунт':['аккаунт', 'регистрация/коды'],
    'доставка_общее':['долгое_ожидание_доставки', 'доставка_общее'],
    'глюки_баги_тормоза':['uxui', 'глюки_баги_тормоза'],
    'лояльность':['купоны','лояльность']
}
synames = []
for i, v in syn.items():
    synames.extend(v)
synames = set(synames)

# no synonim, then problem is synonim to itself
for i in list(clnames-synames):
    syn[i] = [i]


# in loop we find next argmax probability in prediction if it is above threshold
# if no second class, then this label is set to class 'other'
found_labels = []
for ind, row in rvc.iterrows():
    
    di = {}
    for syn_word, syn_group in syn.items():
        idx = [cls2idx[i] for i in syn[syn_word]]
        di[syn_word] = sum(row['y_pred'][idx])

    sorted_di = sorted(di.items(), key=lambda kv: kv[1], reverse=True)  
    
    ixs = [i if v > thresh else 'другое' for i, v in sorted_di][:n]
    for l in range(n-len(ixs)):
        ixs.append('другое')
    ixs[0] = idx2cls[np.argmax(np.array(row['y_pred']))]
    found_labels.append(ixs)

# leave meaningful columns
rvc[[f'found_label_{i}' for i in range(n)]] = pd.DataFrame(found_labels, index=rvc.index)
rvc.drop(columns=['y_pred', 'y_label', 'target'], inplace=True)

# loop through synonims in each row to find second true class
cols = [f'found_label_{i}' for i in range(n)]
t = rvc.query('found_label_0 != "другое" or found_label_1 != "другое"').copy()
secondary_cls = []
for ind, row in t.iterrows():
    true = row['true']
    syns = [true]
    for i, syn_group in syn.items():
        if true in syn_group:
            syns += syn_group
    candidates = [i for i in row[cols] if i != 'другое' and i not in syns]
    secondary_cls.append(candidates[0] if len(candidates) > 0 else np.nan)
t['true_2'] = secondary_cls    


print(t.query('true_2.notnull()', engine='python').shape[0])
t.query('true_2.notnull()', engine='python')

2320


,content,true,found_label_0,found_label_1,true_2
1,Время от времени появляется уведомление о личн...,купоны,купоны,глюки_баги_тормоза,глюки_баги_тормоза
2,"Заказал, оплатил, Готовиться, через 20 минут о...",не_возвращаются_деньги_отмененного_заказа,не_возвращаются_деньги_отмененного_заказа,оплата,оплата
3,За бан бравла в России,другое,другое,обслуживание,обслуживание
4,Мне не положили Pepsi черри в обед за 250 поэт...,другое,другое,обслуживание,обслуживание
5,"Периодически виснет, а после обновления совсем...",обновление,обновление,глюки_баги_тормоза,глюки_баги_тормоза
...,...,...,...,...,...
3172,"невозможно залогиниться, не приходит звонок. с...",регистрация/коды,регистрация/коды,обслуживание,обслуживание
3174,"Сначало был косяк с суммой, теперь с доставкой...",доставка_общее,доставка_общее,обслуживание,обслуживание
3175,"Что за безобразие, заказала на адрес не привез...",доставка_общее,доставка_общее,создание_заказа,создание_заказа
3176,Ок,другое,другое,обслуживание,обслуживание


# explore random comment

In [22]:
s = 'очень неудобное приложение! кое как догадался как оформить доставку! курьер шёл почти час про**ал все сроки, и я его вокруг дома бегал искал. пипец конченые! где оставить отзыв тоже не найти! вы хоть в другие приложения посмотрите я для интереса скачал кфс и сразу на первой стр. написано "доставка или ресторан" может вам хотя-бы сделать такое же простое и понятное приложение?'

t.query('content == @s')

,content,true,found_label_0,found_label_1,true_2
2343,очень неудобное приложение! кое как догадался ...,долгое_ожидание_доставки,долгое_ожидание_доставки,глюки_баги_тормоза,глюки_баги_тормоза


# explore random comments

In [23]:
for ind, row in t.query('true_2.notnull()', engine='python').sample(15).iterrows():
    print(30*'-')
    print(row['content'])
    print()
    print('human label:',row['true'])
    print('suggested label:',row['true_2'])
    print()

------------------------------
Отменили заказ за 15 минут до доставки, без объяснения причин, не рекомендую, больше заказывать не буду. У мака таких косяков не было ни разу. Решил попробовать новинки, остался без ужина

human label: доставка_общее
suggested label: оплата

------------------------------
Ресторан через дорогу, а доставки до меня нет. Посылает на сайт для заказа. Заходишь на сайт, пытаешься пойти авторизацию, а код смс не приходит .

human label: регистрация/коды
suggested label: доставка_общее

------------------------------
После обновления пользоваться приложением стало заметно труднее. Набрал заказ, переходишь к оплате - фиг.

human label: оплата
suggested label: обновление

------------------------------
При попытке заказа из приложения, на этапе оплаты всегда пишет «ресторан временно недоступен для заказа». Проблема в том что это «временно» длится бесконечно...

human label: создание_заказа
suggested label: оплата

------------------------------
Не показывает чек на

# assign target in multiclass style

In [96]:
targets = []
for ind, row in t.iterrows():
    target = [0.0 for i, v in enumerate(cls2idx)]
    target[cls2idx[row['true']]] = 1
    if row['true_2'] is not np.nan and row['true'] != 'другое':
        target[cls2idx[row['true_2']]] = 1.0
    targets.append(target)
        
t['target'] = targets

t.sample(15)

,content,true,found_label_0,found_label_1,true_2,target
1643,"Я максимально занижу все это, вы не можете отв...",обслуживание,обслуживание,другое,NaN,"[0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
2342,"Убрали ""2 ЗА 200"". Это был единственный купон ...",купоны,купоны,другое,NaN,"[0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0...."
1164,"Не могу привязать карту сбербанка, просто выда...",оплата,оплата,глюки_баги_тормоза,глюки_баги_тормоза,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
581,"цены как в московских ресторанах, качество ниж...",цена,цена,другое,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1797,Отвратительно! Короны не списываются!!!,лояльность,лояльность,обслуживание,обслуживание,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1, 0...."
1830,"Раньше на доставку были новые бургеры ""в самое...",доставка_общее,доставка_общее,обслуживание,обслуживание,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1, 0.0, 0...."
1778,"Последнее время могу сказать, что приложение п...",не_возвращаются_деньги_отмененного_заказа,не_возвращаются_деньги_отмененного_заказа,доставка_общее,доставка_общее,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1248,"Ужасное приложение! Постоянные глюки, сбои, по...",обновление,обновление,глюки_баги_тормоза,глюки_баги_тормоза,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
724,После обновления в заказе меняется количество!...,обновление,обновление,другое,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
810,"В заказ добавил соус, слетел весь заказ и прив...",создание_заказа,создание_заказа,глюки_баги_тормоза,глюки_баги_тормоза,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0...."


In [131]:
t.to_csv('multilabel_v1.csv', index=False)